In [ ]:
pip install datasets evaluate --upgrade

In [ ]:
pip install torchtext

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [ ]:
seed=1234

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic=True

In [ ]:
dataset=datasets.load_dataset("bentrevett/multi30k")

In [ ]:
dataset

In [ ]:
train_data, valid_data, test_data=(dataset["train"], dataset["validation"], dataset["test"])

In [ ]:
spacy.cli.download("de_core_news_sm")

In [ ]:
spacy.cli.download("en_core_web_sm")

In [ ]:
import spacy

In [ ]:
en_nlp=spacy.load("en_core_web_sm")
de_nlp=spacy.load("de_core_news_sm")

In [ ]:
def tokenize_text(text,en_nlp,de_nlp,max_length,lower,sos_token,eos_token):
    en_tokens=[token.text for token in en_nlp.tokenizer(text["en"])][:max_length]
    de_tokens=[token.text for token in de_nlp.tokenizer(text["de"])][:max_length]
    if lower :
        en_tokens=[token.lower() for token in en_tokens]
        de_tokens=[token.lower() for token in de_tokens]
    en_tokens=[sos_token]+ en_tokens+[eos_token]
    de_tokens=[sos_token]+de_tokens+[eos_token]
    return {"en_tokens":en_tokens,"de_tokens":de_tokens}

In [ ]:
max_length=1000
lower=True
sos_token="<sos>"
eos_token="<eos>"
fn_kwargs={
    "en_nlp":en_nlp,
    "de_nlp":de_nlp,
    "max_length":max_length,
    "lower":lower,
    "sos_token":sos_token,
    "eos_token":eos_token
}

train_data=train_data.map(tokenize_text,fn_kwargs=fn_kwargs)
valid_data=valid_data.map(tokenize_text,fn_kwargs=fn_kwargs)
test_data=test_data.map(tokenize_text,fn_kwargs=fn_kwargs)
    

In [ ]:
train_data[0]

In [ ]:
import torchtext

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
min_freq=2
unk_token="<unk>"
pad_token="<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [ ]:
en_vocab = build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)
de_vocab = build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

In [ ]:
len(en_vocab), len(de_vocab)

In [ ]:
en_vocab.get_itos()[:10]

In [ ]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [ ]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [ ]:
en_vocab["The"]

In [ ]:
en_vocab.get_itos()[0]

In [ ]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

In [ ]:
def token_to_num(text,en_vocab,de_vocab):
    en_ids = en_vocab.lookup_indices(text["en_tokens"])
    de_ids = de_vocab.lookup_indices(text["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [ ]:
fn_kwargs={"en_vocab":en_vocab,"de_vocab":de_vocab}

train_data = train_data.map(token_to_num, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(token_to_num, fn_kwargs=fn_kwargs)
test_data = test_data.map(token_to_num, fn_kwargs=fn_kwargs)

In [ ]:
train_data[0]

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [ ]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [ ]:
train_data[0]["en_ids"]

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [ ]:
class Encoder(nn.Module):
    def __init__(self,input_dim,embedding_dim,hidden_dim,n_layers,dropout):
        super().__init__()
        self.hidden_dim=hidden_dim
        self.n_layers=n_layers
        self.embedding=nn.Embedding(input_dim,embedding_dim)
        self.rnn=nn.LSTM(embedding_dim,hidden_dim,n_layers,dropout=dropout)
        self.dropout=nn.Dropout(dropout)

    def forward(self,src):
        embedded=self.dropout(self.embedding(src))
        outputs, (hidden, cell)=self.rnn(embedded)
        return hidden,cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self,output_dim,embedding_dim,hidden_dim,n_layers,dropout):
        super().__init__()
        self.output_dim=output_dim
        self.hidden_dim = hidden_dim
        self.n_layers=n_layers
        self.embedding=nn.Embedding(output_dim,embedding_dim)
        self.rnn=nn.LSTM(embedding_dim,hidden_dim,n_layers,dropout=dropout)
        self.f_out=nn.Linear(hidden_dim,output_dim)
        self.dropout=nn.Dropout(dropout)

    def forward(self,input,hidden,cell):
        input = input.unsqueeze(0)
        embedded=self.dropout(self.embedding(input))
        output, (hidden ,cell)= self.rnn(embedded,(hidden,cell))
        prediction=self.f_out(output.squeeze(0))
        return prediction, hidden ,cell
        

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        self.encoder=encoder
        self.decoder=decoder
        self.device=device
        assert(encoder.hidden_dim==decoder.hidden_dim),"Hidden dimension of encoder and decoder must be equal"
        assert(encoder.n_layers==decoder.n_layers),"Encoder and Decoder must have equal number of layers"

    def forward(self,src,trg,teacher_forcing_ratio):
        batch_size=trg.shape[1]
        trg_length=trg.shape[0]
        trg_vocab_size=self.decoder.output_dim
        outputs=torch.zeros(trg_length,batch_size,trg_vocab_size).to(self.device)
        hidden,cell=self.encoder(src)
        input=trg[0,:]
        for t in range(1,trg_length):
            output,hidden,cell=self.decoder(input,hidden,cell)
            outputs[t]=output
            teacher_force=random.random()<teacher_forcing_ratio
            top=output.argmax(1)
            input = trg[t] if teacher_force else top
        return outputs


In [ ]:
input_dim=len(de_vocab)
output_dim=len(en_vocab)
encoder_embedding_dim=256
decoder_embedding_dim=256
hidden_dim=512
n_layers=2
encoder_dropout=0.5
decoder_dropout=0.5
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder=Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout
)

decoder=Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout
)

model=Seq2Seq(encoder,decoder,device).to(device)

In [ ]:
def init_weights(use):
    for name,param in use.named_parameters():
        nn.init.uniform_(param.data,-0.08,0.08)

model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

In [ ]:
optimizer=optim.Adam(model.parameters())
criterion=nn.CrossEntropyLoss(ignore_index=pad_index)

In [ ]:
def train(model,data_loader,optimizer,criterion,clip,teacher_forcing_ratio,device):
    model.train()
    epoch_loss=0
    for i, batch in enumerate(data_loader):
        print(f"Batch {i}")
        src= batch["de_ids"].to(device)
        trg= batch["en_ids"].to(device)
        optimizer.zero_grad()
        output=model(src,trg,teacher_forcing_ratio)
        output_dim=output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg=trg[1:].view(-1)
        loss=criterion(output,trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            print(f"Batch {i}")
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
import tqdm

In [ ]:
n_epochs = 1
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

In [ ]:
print(len(train_data_loader))

In [ ]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

In [ ]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [ ]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [ ]:
translation

In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
references = [[example["en"]] for example in test_data]

In [ ]:
translations = [
    translate_sentence(
        example["de"],
        model,
        en_nlp,
        de_nlp,
        en_vocab,
        de_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_data)
]

In [ ]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["en"]] for example in test_data]

In [ ]:
predictions[0], references[0]

In [ ]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [ ]:
tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

In [ ]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [ ]:
results